<a href="https://colab.research.google.com/github/anjunjin/PFE-ING3-IA/blob/branch1/GetCsvFromJsonOnline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import urllib.request, json 
with urllib.request.urlopen("https://www.lemonde.fr/webservice/decodex/updates") as url: # Obtenir le fichier JSON de https://www.lemonde.fr/webservice/decodex/updates
    data = json.loads(url.read().decode())

# data.keys() #On obtient le premier dictionnaire, sites et url, contient des dictionnaire
# data["sites"].keys() #On obtient le sous-dictionnaire, chaque site, contient des tableaux

In [49]:
# print(data)
# data.keys() # Vérifie la forme que prend les fichier JSON
# data["sites"]
# data["sites"]['1980']
# data["sites"]['1980'][2]
# data["sites"].keys()
# for i in data["sites"].keys():
#   print(type(i))

# for i in data["sites"].keys():
#   print(data["sites"][i][2])

# data["urls"].keys()
# data["urls"]['the-postillon.com']

In [3]:
import pandas as pd

dataset = []

for i in data["sites"].keys(): # On créé un tableau avec la liste de tous les sites du fichier JSON. On a leur identifiant, leur niveau de fiabilité, une description, et 2 noms dont l'un est simplifié
  # dataset.append([int(i),data["sites"][i][0], data["sites"][i][1], data["sites"][i][2], data["sites"][i][3]])
  dataset.append([int(i),data["sites"][i][0], data["sites"][i][1]])

# tableauSites = pd.DataFrame(dataset, columns=['Identifiant','Fiabilité','Description','Nom1','Nom2']) #Pour que le tableau prenne la forme d'un tableau (dataset n'étant en réalité qu'une liste)
tableauSites = pd.DataFrame(dataset, columns=['Identifiant','Fiabilité','Description'])
tableauSites

,Identifiant,Fiabilité,Description
0,1982,2,France-Soir était un journal fondé par Pierre ...
1,1981,2,"Thierry Casasnovas, vidéaste et animateur de l..."
2,1980,2,Silvano Trotta est le créateur d’une société s...
3,1979,2,"Jasper Mader est une chaîne YouTube, animée pa..."
4,1978,2,Une webtélé suisse créée notamment par Chloé F...
...,...,...,...
1338,577,4,Dailymotion n'est pas une source d'information...
1339,576,4,Une plateforme participative qui n'est pas ori...
1340,575,4,9gag n'est pas une source d'information en soi...
1341,574,4,L'encyclopédie collaborative en ligne compile ...


In [4]:
import re

dt = [] #Pour déterminer toutes les lignes dans le JSON qui ne sont pas des liens twitter
compteur = 0 #On note le numéro de chaque ligne qui n'est pas un lien twitter
for i in data["urls"].keys():
  if not(re.search('twitter', i)):
    dt.append(compteur)
  compteur = compteur+1

#On met tous les liens dans df avec des tuples (identifiant, lien)
df = [(data["urls"][i], i) for i in data["urls"].keys()]

#On retire les lignes ne contenant pas de lien twitter, on utilise dt pour ça
for index in sorted(dt, reverse=True):
    del df[index]

#append à df des tuples (identifiant, "NaN") pour tous les identifiants sans lien Twitter
for site in dataset:
  if site[0] not in [item for t in df for item in t]:
    df.append((site[0], "NaN")) #Objectif : Obtenir liste users avec Nom2 et comparaison NLP grâce à Description -> "twitter.com/"+username
df.sort(reverse=True)

#On rajoute la nouvelle colonne ne contenant que les liens au tableau
tableauSites['Twitter'] = [item[1] for item in df]

tableauSites

,Identifiant,Fiabilité,Description,Twitter
0,1982,2,France-Soir était un journal fondé par Pierre ...,twitter.com/france_soir
1,1981,2,"Thierry Casasnovas, vidéaste et animateur de l...",twitter.com/ThierryRegenere
2,1980,2,Silvano Trotta est le créateur d’une société s...,twitter.com/silvano_trotta
3,1979,2,"Jasper Mader est une chaîne YouTube, animée pa...",NaN
4,1978,2,Une webtélé suisse créée notamment par Chloé F...,NaN
...,...,...,...,...
1338,577,4,Dailymotion n'est pas une source d'information...,NaN
1339,576,4,Une plateforme participative qui n'est pas ori...,twitter.com/agoravox
1340,575,4,9gag n'est pas une source d'information en soi...,twitter.com/9GAG
1341,574,4,L'encyclopédie collaborative en ligne compile ...,twitter.com/Wikipedia


#dans dataset remove lignes sans lien Twitter (inutiles) puis chaque ligne prendre nom exact avec lien Twitter et obtenir X tweets (200 par exemple) puis analyse de thème dessus.

In [11]:
import copy
save_dataset = copy.deepcopy(dataset)

i = 0
for i in range(len(save_dataset)):
  save_dataset[i] = save_dataset[i] + [df[i][1]]
  i += 1

save_dataset = [x for x in save_dataset if x[3] != "NaN"] #Retire les lignes où pas de lien
for line in save_dataset:
  if "twitter.com/" in line[3]:
    if "?lang=fr" in line[3]:
      size = len(line[3])
      line[3] = line[3][12:size - 8].strip('@')
    else:
      line[3] = line[3][12:].strip('@')
# save_dataset = [[x[0],x[1],x[2],x[3][12:len(x[3]) - 8]] for x in save_dataset if "?lang=fr" in x[3]:] #Retire les lignes où pas de lien

save_dataset

tableauSites = pd.DataFrame(save_dataset, columns=['Identifiant','Fiabilité','Description','Twitter'])
tableauSites

,Identifiant,Fiabilité,Description,Twitter
0,1982,2,France-Soir était un journal fondé par Pierre ...,france_soir
1,1981,2,"Thierry Casasnovas, vidéaste et animateur de l...",ThierryRegenere
2,1980,2,Silvano Trotta est le créateur d’une société s...,silvano_trotta
3,1976,2,Cette propriétaire d’un magasin de chaussures ...,emakrusi
4,1973,2,Salim Laïbi est un chirurgien-dentiste connu s...,llp_le_vrai
...,...,...,...,...
420,583,4,"Attention, il ne s'agit pas d'articles du « Fi...",FigaroVox
421,576,4,Une plateforme participative qui n'est pas ori...,agoravox
422,575,4,9gag n'est pas une source d'information en soi...,9GAG
423,574,4,L'encyclopédie collaborative en ligne compile ...,Wikipedia


In [65]:
re.split('twitter.com/', 'twitter.com/silvano_trotta')

['', 'silvano_trotta']

In [55]:
# tableauSites.to_csv('json1.csv')
# !cp json1.csv "/content/drive/My Drive/PFE"

In [56]:
# 1 : site parodique
# 2 : non fiable
# 3 : site à la fiabilité douteuse
# 4 : fiable